# LSTM Model 3 on  Covid-19 dataset 

Here you can see a multi-input (title and text column) model was created, where the two previous architectures (LSTM1 and LSTM2) are merged in one. 
This model is done by Andrii Shchur, 2020
Here is its [article](https://towardsdatascience.com/fake-news-detector-with-deep-learning-approach-part-i-eda-757f5c052)
![image.png](../../images/model3.png)

FakeCovid is the first multilingual cross-domain dataset of 7623 fact-checked news articles for COVID-19, collected from 04/01/2020 to 01/07/2020. We have collected the fact-checked articles from 92 fact-checking websites after obtaining references from Poynter and Snopes. We have manually annotated the collected articles into 11 categories of the fact-checked news according to their content. The ultimately generated dataset is in 40 languages from 105 countries.

<img src='https://media.giphy.com/media/RiVgwsIl1CZAcFQA88/giphy.gif'>

### <span style="background-color: #FFFF00">In this notebook, I have kept the orogonal validation techenique and the oroginal preprocessing of the dataset </span>

## Importing packages 

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
import time
#Loading tensorflow and keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import plot_model
from tensorflow.keras.layers import Input
#Loading ntlk
import nltk
import re
from nltk.corpus import stopwords #corpus is collection of text
from nltk.stem.porter import PorterStemmer
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Loading the model 
import sys  
sys.path.insert(0, '../../models')
from PY.model3 import get_model

## Paths 

In [20]:
DATA_DIR = '../../../../Data'
CODE_DIR_H5='../../models/H5'
# Create a path for the saving location of the model
MODEL_DIR_H5 = CODE_DIR_H5 + '/model3_Covid-19.h5'

## Loading data 

In [21]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'Covid-19/Claims_to_work_with/claims.csv'))
train.head()

label                                              title  \
0      1   Chinese converting to Islam after realizing t...   
1      1   Captions on a reuploaded video about the U.S....   
2      0   Ghana has 307 ambulances with mobile ventilat...   
3      1   “Governor Andy Beshear has authorized Kentuck...   
4      1   Photo shows food being distributed to Rwandan...   

                                                text  
0  The fact behind every news!, Ever since the Wo...  
1  AUTHOR, VERA Files, DATE, May 08, 2020, SHARE,...  
2  Source: Dr Anthony Nsiah Asare, Verdict: Mostl...  
3  More Info, Trying to focus on school work at h...  
4  A photo of hundreds of neat piles of bedding a...

In [22]:
# get label
X_train=train.drop('label',axis=1)
y_train=train['label']

In [23]:
# set vocabulary size
vo_size=500
messages=X_train.copy()
messages.reset_index(inplace=True)

In [24]:
#dataset Preprocessing
ps_title =PorterStemmer()
ps_text =PorterStemmer()
corpus_title = []
corpus_text = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    
    #preproc title
    review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps_title.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_title.append(review)
    
    #preproc text
    review = re.sub('[^a-zA-Z]', ' ',messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps_text.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_text.append(review)

In [25]:
# One hot representation
onehot_rep_title = [one_hot(words, vo_size) for words in corpus_title]
onehot_rep_text = [one_hot(words, vo_size) for words in corpus_text]

In [26]:
# sequences representation
sent_length_title = 20
sent_length_text = 1000
embedded_doc_title=pad_sequences(onehot_rep_title, padding='pre', maxlen=sent_length_title)
embedded_doc_text=pad_sequences(onehot_rep_text, padding='pre', maxlen=sent_length_text)

In [27]:
# check shape 
print(len(embedded_doc_title),y_train.shape)
print(len(embedded_doc_text),y_train.shape)

4608 (4608,)
4608 (4608,)


In [28]:
# final data for NN
X_final_title=np.array(embedded_doc_title)
X_final_text=np.array(embedded_doc_text)
y_final=np.array(y_train)
print(X_final_title.shape,y_final.shape)
print(X_final_text.shape,y_final.shape)

(4608, 20) (4608,)
(4608, 1000) (4608,)


In [29]:
embedding_vector_feature_title = 10
embedding_vector_feature_text = 100
input_title = Input(shape=(sent_length_title,))
input_text = Input(shape=(sent_length_text,))

In [ ]:
start=time.time()
model=get_model(vo_size,embedding_vector_feature_title,embedding_vector_feature_text,input_title,input_text)
history = model.fit(x=[X_final_title,X_final_text], y=y_final, batch_size=128, epochs=10, verbose=1, validation_split=0.2)
scores = model.evaluate(x=[X_final_title,X_final_text], y=y_final, verbose=0)
end=time.time()

In [ ]:
# Save the model
model.save(MODEL_DIR_H5)
model.summary()

In [ ]:
print("accuracy","%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("precision","%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
print("recall","%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
print("f1-mesure","%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))

In [ ]:
print('time of training' , end-start)

## Graphs 

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for f1-mesure
plt.plot(history.history['f1_mesure'])
plt.title('model F1-mesure')
plt.ylabel('F1-mesure')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for precision
plt.plot(history.history['precision_mesure'])
plt.title('model precision')
plt.ylabel('precision')
plt.xlabel('epoch')
plt.show()

In [ ]:
# summarize history for recall
plt.plot(history.history['recall_mesure'])
plt.title('model recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.show()